In [6]:
import requests
from io import StringIO
import pandas as pd
import datetime
import numpy as np

In [7]:
def chipCrawler(csvFileName, dateList):
    # csvFileName : means "價量資料庫.csv" in my case.
    # dateList : for example, ["20191225","20191226","20191227","20191230","20191231"], nearby 5 days.
    # data source : TWSE&TPEX
    dfPriceData = pd.read_csv(csvFileName, encoding = "utf-8")
    SecurityList = sorted(set([i for i in dfPriceData["證券代號"]]))
    
    # 上市－三大法人動向
    for i in dateList:
        r = requests.get('http://www.tse.com.tw/fund/T86?response=csv&date=' + i + '&selectType=ALLBUT0999') #不含權證、牛熊證
        df = pd.read_csv(StringIO(r.text), header=1).dropna(how='all', axis=1).dropna(how='any')
        df["日期"] = int(i)
        if i == dateList[0]:
            dfChip = df
        else:
            dfChip = pd.concat([dfChip,df], ignore_index=True, sort=False)
    dfChip = dfChip.sort_values(by = ["證券代號","日期"])
    dfChip.reset_index(drop = True, inplace = True)
    
    # 上櫃－三大法人動向
    otcdatestr = [str(int(datetime.datetime.strftime(datetime.datetime.strptime(i, "%Y%m%d"), "%Y/%m/%d").split("/")[0])-1911) + "/" + datetime.datetime.strftime(datetime.datetime.strptime(i, "%Y%m%d"), "%Y/%m/%d").split("/")[1] + "/" + datetime.datetime.strftime(datetime.datetime.strptime(i, "%Y%m%d"), "%Y/%m/%d").split("/")[2] for i in dateList]
    for i in otcdatestr:
        r = requests.get("https://www.tpex.org.tw/web/stock/3insti/daily_trade/3itrade_hedge_download.php?l=zh-tw&se=EW&t=D&d=" + i + "&s=0,asc,0") #不含權證、牛熊證
        df = pd.read_csv(StringIO(r.text), header=1)
        df["日期"] = int(str(int(i.split("/")[0])+1911) + i.split("/")[1] + i.split("/")[2])
        if i == otcdatestr[0]:
            dfChip1 = df
        else:
            dfChip1 = pd.concat([dfChip1,df], ignore_index=True, sort=False)
    dfChip1 = dfChip1.sort_values(by = ["代號","日期"])
    dfChip1.reset_index(drop = True, inplace = True)
    dfChip1.drop(columns=['外資及陸資-買進股數', '外資及陸資-賣出股數', '外資及陸資-買賣超股數','自營商-買進股數', '自營商-賣出股數'], inplace=True)
    dfChip1 = dfChip1[['代號', '名稱', '外資及陸資(不含外資自營商)-買進股數', '外資及陸資(不含外資自營商)-賣出股數',
       '外資及陸資(不含外資自營商)-買賣超股數', '外資自營商-買進股數', '外資自營商-賣出股數', '外資自營商-買賣超股數',
       '投信-買進股數', '投信-賣出股數', '投信-買賣超股數', '自營商-買賣超股數', '自營商(自行買賣)-買進股數', '自營商(自行買賣)-賣出股數',
       '自營商(自行買賣)-買賣超股數', '自營商(避險)-買進股數', '自營商(避險)-賣出股數', '自營商(避險)-買賣超股數',
       '三大法人買賣超股數合計', '日期']]
    for i in range(20):
        dfChip1.rename(columns = {dfChip1.columns[i]:dfChip.columns[i]} ,inplace = True)
    
    #合併為上市上櫃－三大法人動向
    dfLegalPerson = pd.concat([dfChip,dfChip1], ignore_index=True, sort=False)
    dfLegalPerson['證券代號'] = [i.replace("=","").replace("\"","")for i in dfLegalPerson['證券代號']]
    
    # 上市－券資概況，可進一步算出券資比。
    for i in dateList:
        r = requests.get("https://www.twse.com.tw/exchangeReport/MI_MARGN?response=csv&date=" + i + "&selectType=ALL")
        rtext = r.text.replace(r.text.split(",,,,,,,,")[0]+",,,,,,,,\r\n","")
        df = pd.read_csv(StringIO(rtext)).dropna(how='all', axis=1).dropna(how='any')
        df["日期"] = int(i)
        if i == siidatestr[0]:
            dfChip2 = df
        else:
            dfChip2 = pd.concat([dfChip2,df], ignore_index=True, sort=False)
    dfChip2 = dfChip2.sort_values(by = ["股票代號","日期"])
    dfChip2.reset_index(drop = True, inplace = True)
    dfChip2['股票代號'] = [i.replace("=","").replace("\"","") for i in dfChip2["股票代號"]]
    dfChip2.rename(columns = {'買進':"資買",'賣出':"資賣",'現金償還':"現償",'前日餘額':"前日資餘",'今日餘額':"今日資餘",'限額':"資限額",
                             '買進.1':"券買",'賣出.1':"券賣",'現券償還':"券償",'前日餘額.1':"前日券餘",'今日餘額.1':"今日券餘",'限額.1':"券限額",} ,inplace = True)
    
    # 上櫃－券資概況，可進一步算出券資比。
    for i in otcdatestr:
        r = requests.get("https://www.tpex.org.tw/web/stock/margin_trading/margin_balance/margin_bal_download.php?l=zh-tw&d=" + i + "&s=0,asc,0")
        rtext = r.text.replace(r.text.split("代號")[0],"")
        df = pd.read_csv(StringIO(rtext))
        df["日期"] = int(str(int(i.split("/")[0])+1911) + i.split("/")[1] + i.split("/")[2])
        if i == otcdatestr[0]:
            dfChip3 = df
        else:
            dfChip3 = pd.concat([dfChip3,df], ignore_index=True, sort=False)
    dfChip3 = dfChip3.sort_values(by = ["代號","日期"])
    dfChip3.reset_index(drop = True, inplace = True)
    remove = []
    for i in range(len(dfChip3)):
        if dfChip3['代號'][i].isdigit() == False:
            remove.append(i)
    dfChip3.drop(index=remove, inplace=True)
    dfChip3.reset_index(drop = True, inplace = True)
    dfChip3.drop(columns=['  資屬證金',' 資使用率(%)','  券屬證金','  券使用率(%)'], inplace=True)
    dfChip3.rename(columns = {'代號':"股票代號",'  名稱':"股票名稱",'  前資餘額(張)':"前日資餘","  現償":"現償",'  資餘額':"今日資餘",' 資限額':"資限額",
                             ' 前券餘額(張)':"前日券餘",'   券賣':"券賣",'  券買':"券買",'  券償':"券償",'  券餘額':"今日券餘",' 券限額':"券限額","  資券相抵(張)":"資券互抵",
                             '  備註 ':"註記",'  資賣':"資賣",'  資買':"資買",} ,inplace = True)
    dfChip3 = dfChip3[['股票代號', '股票名稱', '資買', '資賣', '現償', '前日資餘', '今日資餘', '資限額', '券買', '券賣',
           '券償', '前日券餘', '今日券餘', '券限額', '資券互抵', '註記', '日期']]
    
    # 合併為上市上櫃－券資概況
    dfMarginShort = pd.concat([dfChip2,dfChip3], ignore_index=True, sort=False)
    
    # 在券資資料，卻沒有價量資料的股票代碼們。
    retMarginShort = list(set(list(dfMarginShort["股票代號"])).difference(set(SecurityList)))
    # 在三大法人資料，卻沒有價量資料的股票代碼們。
    retLegalPerson = list(set(list(dfLegalPerson["證券代號"])).difference(set(SecurityList)))
    # 清理這些資料
    for i in retMarginShort:
        removeIndex = dfMarginShort[dfMarginShort["股票代號"]==i].index.tolist()
        dfMarginShort.drop(index=removeIndex, inplace=True)
    dfMarginShort.reset_index(drop = True, inplace = True)
    for i in retLegalPerson:
        removeIndex = dfLegalPerson[dfLegalPerson["證券代號"]==i].index.tolist()
        dfLegalPerson.drop(index=removeIndex, inplace=True)
    dfLegalPerson.reset_index(drop = True, inplace = True)
    
    # 券資資料多的
    ret1 = list(set(list(dfMarginShort["股票代號"])).difference(set(list(dfLegalPerson["證券代號"]))))
    # 三大法人資料多的
    ret2 = list(set(list(dfLegalPerson["證券代號"])).difference(set(list(dfMarginShort["股票代號"]))))
    for i in ret1:
        removeIndex = dfMarginShort[dfMarginShort["股票代號"]==i].index.tolist()
        dfMarginShort.drop(index=removeIndex, inplace=True)
    dfMarginShort.reset_index(drop = True, inplace = True)
    for i in ret2:
        removeIndex = dfLegalPerson[dfLegalPerson["證券代號"]==i].index.tolist()
        dfLegalPerson.drop(index=removeIndex, inplace=True)
    dfLegalPerson.reset_index(drop = True, inplace = True)
    
    # 三大人資料會比券資資料少，主要是因為不是每檔股票天天都有三大法人做操作，導致不會都有資料。
    # 填補dfLegalPerson的資料，用o取代！
    intervalDict = {}
    for key in dfLegalPerson['證券代號']:
        intervalDict[key] = intervalDict.get(key,0) + 1
    addDict = {}
    for i in intervalDict:
        if intervalDict[i] < 5:
            addDict[i] = intervalDict[i]
    for i in addDict:
        df = dfLegalPerson[dfLegalPerson["證券代號"] == i]
        dateList = list(df["日期"])
        ret = list(set([int(i) for i in siidatestr]).difference(set(dateList)))
        for j in ret:
            dfLegalPerson.loc['newraw'+str(i)+str(j)] = '0'
            dfLegalPerson['證券代號']['newraw'+str(i)+str(j)] = i
            dfLegalPerson["日期"]['newraw'+str(i)+str(j)] = j
    #完成填補
    dfLegalPerson = dfLegalPerson.sort_values(by = ["證券代號","日期"])
    dfLegalPerson.reset_index(drop = True, inplace = True)
    
    # 進行合併
    dfMarginShort["前日資餘"] = [int(i.replace(",","")) for i in dfMarginShort["前日資餘"]]
    dfMarginShort["今日資餘"] = [int(i.replace(",","")) for i in dfMarginShort["今日資餘"]]
    dfMarginShort["前日券餘"] = [int(i.replace(",","")) for i in dfMarginShort["前日券餘"]]
    dfMarginShort["今日券餘"] = [int(i.replace(",","")) for i in dfMarginShort["今日券餘"]]
    dfMarginShort["資增"] = dfMarginShort['今日資餘'] - dfMarginShort['前日資餘']
    dfMarginShort["券增"] = dfMarginShort['今日券餘'] - dfMarginShort['前日券餘']
    dfMarginShort["券資比(%)"] = round((dfMarginShort['今日券餘']/dfMarginShort['今日資餘']), 4) * 100

    dfMarginShort["外陸資買賣超股數(不含外資自營商)"] = dfLegalPerson['外陸資買賣超股數(不含外資自營商)']
    dfMarginShort["外資自營商買賣超股數"] = dfLegalPerson['外資自營商買賣超股數']
    dfMarginShort["投信買賣超股數"] = dfLegalPerson['投信買賣超股數']
    dfMarginShort["自營商買賣超股數(自行買賣)"] = dfLegalPerson['自營商買賣超股數(自行買賣)']
    dfMarginShort["自營商買賣超股數(避險)"] = dfLegalPerson['自營商買賣超股數(避險)']
    dfMarginShort["三大法人合計買賣超"] = dfLegalPerson['三大法人買賣超股數']
    dfMarginShort.drop(columns=["註記"], inplace=True)
    dfMarginShort.rename(columns = {'股票代號':"證券代號",'股票名稱':"證券名稱"} ,inplace = True)
    return dfMarginShort.to_csv('籌碼資料庫.csv', encoding="utf-8", index = 0)